In [3]:
import pandas as pd
import numpy as np

raw = pd.read_csv(
    r"C:\credit_portfolio_analysis\Data\raw\lending_club_raw.csv"
, low_memory = False)

print("Shape:", raw.shape)
print('\n No.of columns:', len(raw.columns))
print('\nfirst 5 cols:')
print(raw.columns[:5])
print("\nLast 5 columns:")
print(raw.columns[-5:])
raw.head()

Shape: (2260668, 145)

 No.of columns: 145

first 5 cols:
Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv'], dtype='object')

Last 5 columns:
Index(['settlement_status', 'settlement_date', 'settlement_amount',
       'settlement_percentage', 'settlement_term'],
      dtype='object')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
raw[['id', 'member_id']].isna().mean()


id           1.0
member_id    1.0
dtype: float64

In [26]:
raw.duplicated().sum()

np.int64(0)

In [27]:
(raw['loan_amnt'] <= 0).sum()

np.int64(0)

In [29]:
raw['loan_amnt'].describe()

count    2.260668e+06
mean     1.504693e+04
std      9.190245e+03
min      5.000000e+02
25%      8.000000e+03
50%      1.290000e+04
75%      2.000000e+04
max      4.000000e+04
Name: loan_amnt, dtype: float64

In [30]:
raw['loan_status'].value_counts()

loan_status
Fully Paid                                             1041952
Current                                                 919695
Charged Off                                             261655
Late (31-120 days)                                       21897
In Grace Period                                           8952
Late (16-30 days)                                         3737
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     31
Name: count, dtype: int64

In [32]:
rn = raw.copy()

In [34]:
rn['loan_id'] = rn.index

In [35]:
rn.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,loan_id
0,NaN,NaN,2500,2500,2500.0,36 months,13.56,84.92,C,C1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,30000,30000,30000.0,60 months,18.94,777.23,D,D2,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NaN,NaN,5000,5000,5000.0,36 months,17.97,180.69,D,D1,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,NaN,4000,4000,4000.0,36 months,18.94,146.51,D,D2,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,3
4,NaN,NaN,30000,30000,30000.0,60 months,16.14,731.78,C,C4,...,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,4


In [36]:
rn['loan_lifecycle'] = np.where(rn['loan_status'] == 'Fully Paid', 'Closed','Open')

In [37]:
rn['loan_lifecycle'].value_counts()

loan_lifecycle
Open      1218716
Closed    1041952
Name: count, dtype: int64

In [39]:
rn.groupby('loan_lifecycle').agg(loan_count = ('loan_id' , 'count') , total_exposure = ('loan_amnt' , 'sum'))

,loan_count,total_exposure
loan_lifecycle,,
Closed,1041952,14725380075
Open,1218716,19290735850


In [44]:
# STEP 3: DPD BUCKET CREATION

# 1. Initialize dpd_bucket with a default value
rn['dpd_bucket'] = 'Other'

# 2. Closed loans (out of risk)
rn.loc[rn['loan_status'] == 'Fully Paid','dpd_bucket'] = 'Closed'

# 3. Current loans (0 DPD)
rn.loc[rn['loan_status'] == 'Current', 'dpd_bucket'] = 'Current'

# 4. Early delinquency (1–15 DPD)
rn.loc[rn['loan_status'] == 'In Grace Period', 'dpd_bucket'] = 'DPD_1_15'

# 5. PAR 30 bucket (16–30 DPD)
rn.loc[rn['loan_status'] == 'Late (16-30 days)', 'dpd_bucket'] = 'DPD_16_30'

# 6. PAR 60/90 proxy (31–120 DPD)
rn.loc[rn['loan_status'] == 'Late (31-120 days)', 'dpd_bucket'] = 'DPD_31_90'

# 7. Severe delinquency / default (90+ DPD)
rn.loc[
    rn['loan_status'].isin(['Charged Off', 'Default']),
    'dpd_bucket'
] = 'DPD_90_plus'

# 8. Final validation checks
rn['dpd_bucket'].value_counts()


dpd_bucket
Closed         1041952
Current         919695
DPD_90_plus     261686
DPD_31_90        21897
DPD_1_15          8952
DPD_16_30         3737
Other             2749
Name: count, dtype: int64

In [45]:
rn.loc[rn['dpd_bucket'] == 'Other', 'loan_status'].value_counts()

loan_status
Does not meet the credit policy. Status:Fully Paid     1988
Does not meet the credit policy. Status:Charged Off     761
Name: count, dtype: int64

In [49]:
rn.loc[
    rn['loan_status'] == 'Does not meet the credit policy. Status:Fully Paid',
    'dpd_bucket'
] = 'Closed'

rn.loc[
    rn['loan_status'] == 'Does not meet the credit policy. Status:Charged Off',
    'dpd_bucket'
] = 'DPD_90_plus'

rn['dpd_bucket'].value_counts()


dpd_bucket
Closed         1043940
Current         919695
DPD_90_plus     262447
DPD_31_90        21897
DPD_1_15          8952
DPD_16_30         3737
Name: count, dtype: int64

In [50]:
rn.loc[rn['dpd_bucket'] == 'Other', 'loan_status'].value_counts()


Series([], Name: count, dtype: int64)

In [67]:
risk_df = rn[rn['dpd_bucket'] != 'Closed']
risk_df.groupby('loan_term_months')['loan_amnt'].sum()


loan_term_months
36    10102796525
60     9170339100
Name: loan_amnt, dtype: int64

In [70]:
total_exposure_by_term = (
    risk_df.groupby('loan_term_months')['loan_amnt'].sum()
)
total_exposure_by_term


loan_term_months
36    10102796525
60     9170339100
Name: loan_amnt, dtype: int64

In [55]:
par30_buckets = ['DPD_16_30', 'DPD_31_90', 'DPD_90_plus']
par30_exposure = risk_df.loc[
    risk_df['dpd_bucket'].isin(par30_buckets),
    'loan_amnt'
].sum()
par30_ratio = par30_exposure / total_active_exposure
par30_ratio



np.float64(0.2337808251686601)

In [58]:
par60_exposure = risk_df.loc[
    risk_df['dpd_bucket'].isin(['DPD_31_90', 'DPD_90_plus']),
    'loan_amnt'
].sum()

par60_ratio = par60_exposure / total_active_exposure
par60_ratio


np.float64(0.2304878283136141)

In [71]:
total_exposure_by_term = (
    risk_df
    .groupby('loan_term_months')['loan_amnt']
    .sum()
)

total_exposure_by_term


loan_term_months
36    10102796525
60     9170339100
Name: loan_amnt, dtype: int64

In [72]:
par90_exposure = risk_df.loc[
    risk_df['dpd_bucket'] == 'DPD_90_plus',
    'loan_amnt'
].sum()

par90_ratio = par90_exposure / total_active_exposure
par90_ratioterm_risk_summary = pd.DataFrame({
    'Total_Exposure': total_exposure_by_term,
    'PAR_30_plus_Exposure': par30_term,
    'PAR_60_plus_Exposure': par60_term,
    'PAR_90_plus_Exposure': par90_term
}).fillna(0)

term_risk_summary


NameError: name 'par30_term' is not defined

In [60]:
par_summary = pd.DataFrame({
    'PAR_Level': ['PAR_30_plus', 'PAR_60_plus', 'PAR_90_plus'],
    'Exposure': [par30_exposure, par60_exposure, par90_exposure],
    'Ratio': [par30_ratio, par60_ratio, par90_ratio]
})

par_summary


,PAR_Level,Exposure,Ratio
0,PAR_30_plus,4505689550,0.233781
1,PAR_60_plus,4442223175,0.230488
2,PAR_90_plus,4076208225,0.211497


In [64]:
rn['loan_term_months'] = rn['term'].str.extract(r'(\d+)').astype(int)
rn['loan_term_months'].value_counts()



loan_term_months
36    1609754
60     650914
Name: count, dtype: int64

In [74]:
term_risk_summary = pd.DataFrame({
    'Total_Exposure': total_exposure_by_term,
    'PAR_30_plus_Exposure': par30_exposure,
    'PAR_90_plus_Exposure': par90_exposure
}).fillna(0)

term_risk_summary


,Total_Exposure,PAR_30_plus_Exposure,PAR_90_plus_Exposure
loan_term_months,,,
36,10102796525,4505689550,4076208225
60,9170339100,4505689550,4076208225


In [75]:
par_summary = pd.DataFrame({
    'PAR_Level': ['PAR_30_plus', 'PAR_60_plus', 'PAR_90_plus'],
    'Exposure': [par30_exposure, par60_exposure, par90_exposure],
    'Ratio': [par30_ratio, par60_ratio, par90_ratio]
})

par_summary


,PAR_Level,Exposure,Ratio
0,PAR_30_plus,4505689550,0.233781
1,PAR_60_plus,4442223175,0.230488
2,PAR_90_plus,4076208225,0.211497


In [76]:
par30_term = (
    risk_df[
        risk_df['dpd_bucket'].isin(['DPD_16_30', 'DPD_31_90', 'DPD_90_plus'])
    ]
    .groupby('loan_term_months')['loan_amnt']
    .sum()
)


In [77]:
par90_term = (
    risk_df[
        risk_df['dpd_bucket'] == 'DPD_90_plus'
    ]
    .groupby('loan_term_months')['loan_amnt']
    .sum()
)


In [78]:
term_risk_summary = pd.DataFrame({
    'Total_Exposure': total_exposure_by_term,
    'PAR_30_plus_Exposure': par30_term,
    'PAR_90_plus_Exposure': par90_term
}).fillna(0)

term_risk_summary


,Total_Exposure,PAR_30_plus_Exposure,PAR_90_plus_Exposure
loan_term_months,,,
36,10102796525,2216237550,2010077525
60,9170339100,2289452000,2066130700


In [79]:
term_risk_summary['PAR_30_ratio'] = (
    term_risk_summary['PAR_30_plus_Exposure'] /
    term_risk_summary['Total_Exposure']
)

term_risk_summary['PAR_90_ratio'] = (
    term_risk_summary['PAR_90_plus_Exposure'] /
    term_risk_summary['Total_Exposure']
)

term_risk_summary


,Total_Exposure,PAR_30_plus_Exposure,PAR_90_plus_Exposure,PAR_30_ratio,PAR_90_ratio
loan_term_months,,,,,
36,10102796525,2216237550,2010077525,0.219369,0.198962
60,9170339100,2289452000,2066130700,0.249658,0.225306
